In [1]:
import sys
sys.path.insert(1, '../../code/scripts')

In [2]:
# analysis
import numpy as np
from scipy.stats import mannwhitneyu
import pandas as pd
import tempparse as paf

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams.update({'font.size': 14})
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = "Arial"

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Setting up

In [4]:
# Calculate scaled hydropathy
kd_scale = {"A": 1.8, "R": -4.5, "N": -3.5, "D": -3.5, "C": 2.5, "E": -3.5,
                      "Q": -3.5, "G": -0.4, "H": -3.2, "I": 4.5, "L": 3.8, "K": -3.9,
                      "M": 1.9, "F": 2.8, "P": -1.6, "S": -0.8, "T": -0.7, "W": -0.9,
                      "Y": -1.3, "V": 4.2}

vals = list(kd_scale.values())
scaled_vals = [v-np.min(vals) for v in vals]


kd_scale_norm = {}
for k in kd_scale.keys():
    kd_scale_norm[k] = (kd_scale[k] - np.min(vals)) / np.max(scaled_vals)

In [5]:
def append_hydropathy(row):
    seq = row['region_seq']
    return paf.calculate_hydropathy(seq, scale=kd_scale_norm)

In [6]:
def append_norm_nc(row):
    seq = row['region_seq']
    nc = np.abs((seq.count('R') + seq.count('K') - seq.count('E') + seq.count('D'))) / row['len_region']
    return nc

In [7]:
def append_uversky_pred(row):
    norm_hydrop = row['norm_hydrop']
    norm_nc = row['norm_nc']
    if norm_nc > (2.785 * norm_hydrop - 1.151):
        return "disordered"
    else:
        return "helix"

In [8]:
def append_uversky_correctness(row):
    if row['label'] == row['uversky_pred']:
        return 1
    else:
        return 0

## Test dataset of LR Model

In [9]:
df = pd.read_csv('../../data/af_regions/sc_af_regions_testing.csv')
df.head()

,start,end,uni_id,region_seq,len_region,label,freq_A,freq_C,freq_D,freq_E,...,freq_M,freq_N,freq_P,freq_Q,freq_R,freq_S,freq_T,freq_V,freq_W,freq_Y
0,171,216,P31385,PIEMEEQRMTALKEITDIEYKFAQLRQKLYDNQLVRLQTELQMCLE,46,helix,0.043478,0.021739,0.043478,0.152174,...,0.065217,0.021739,0.021739,0.130435,0.065217,0.000000,0.065217,0.021739,0.000000,0.043478
1,34,82,P31379,FNQYLDKVYKKHFSKVMSRTRYVLMNFFKDAFTGGAFMYPFKGFLEFNT,49,helix,0.040816,0.000000,0.040816,0.020408,...,0.061224,0.061224,0.020408,0.020408,0.040816,0.040816,0.061224,0.061224,0.000000,0.081633
2,186,214,P31379,IYFWAYYFPFKLVKYMLGLSVLFVLLVIS,29,helix,0.034483,0.000000,0.000000,0.000000,...,0.034483,0.000000,0.034483,0.000000,0.000000,0.068966,0.000000,0.137931,0.034483,0.137931
3,75,106,P31381,LSYITFFAIGIGLLWPWNCILSASQYFKHDIF,32,helix,0.062500,0.031250,0.031250,0.000000,...,0.000000,0.031250,0.031250,0.031250,0.000000,0.093750,0.031250,0.000000,0.062500,0.062500
4,101,128,P10962,KFFRHKCKQRFTKLTQVMITERRLALRE,28,helix,0.035714,0.035714,0.000000,0.071429,...,0.035714,0.000000,0.000000,0.071429,0.178571,0.000000,0.107143,0.035714,0.000000,0.000000


In [10]:
df['norm_hydrop'] = df.apply(lambda row: append_hydropathy(row), axis=1)
df['norm_nc'] = df.apply(lambda row: append_norm_nc(row), axis=1)
df['uversky_pred'] = df.apply(lambda row: append_uversky_pred(row), axis=1)
df['uversky_correctness'] = df.apply(lambda row:append_uversky_correctness(row), axis=1)

In [11]:
df.to_csv('../../data/uversky/uversky_all.csv', index=False)

In [12]:
df = df.sort_values(by='label', ascending=False)

## Random non-highly-charged regions

In [13]:
df_rd = pd.read_csv('../../data/af_regions/random_af_regions_low_thresh.csv')
df_rd.head()

,uni_id,seq,left_bound,right_bound,label
0,Q04007,FYAPSILSRSGSSTDVLSSGIDSMAKNSKETRGRFRSLSLMDPALQ...,503,552,disordered
1,P22516,PIAEVVTLPYQYLLSESTRSSLQINLENSIVIIDEAHNLIETINSI...,359,452,helix
2,Q12218,APSSSEVVSSSVASSSSEVASSSVAPSSSEVVSSSVASSSSEVASS...,164,228,disordered
3,P38196,TIPTEDYEEITKESEMGDATKITSKIDANVIEKKDTDSENNITIAQ...,18,94,disordered
4,Q12186,IQGIVCKICGQTGHFSRDCNSSSQRMSRFDRNATVNNSAPIQSNDV...,293,369,disordered


In [14]:
df_rd = df_rd.rename(columns={'seq': 'region_seq'})

In [15]:
def append_region_len(row):
    return len(row['region_seq'])

In [16]:
df_rd['len_region'] = df_rd.apply(lambda row: append_region_len(row), axis=1)

In [17]:
df_rd['norm_hydrop'] = df_rd.apply(lambda row: append_hydropathy(row), axis=1)
df_rd['norm_nc'] = df_rd.apply(lambda row: append_norm_nc(row), axis=1)
df_rd['uversky_pred'] = df_rd.apply(lambda row: append_uversky_pred(row), axis=1)
df_rd['uversky_correctness'] = df_rd.apply(lambda row:append_uversky_correctness(row), axis=1)

In [18]:
df_rd.to_csv('../../data/uversky/uversky_random.csv', index=False)

## Highly charged regions - trimmed

In [19]:
df_hc = pd.read_csv('../../data/charged_regions/cr_trimmed_filtered_aflabel.csv')
df_hc.head()

,orf,gene,seq.len,left.bound,right.bound,region.seq,region.len,charge.asymmetry,frac.charge,uni_id,orf_label,kappa,label
0,YAL011W,SWC3,626,5,53,RTRSKESSIEQKPASRTRTRSRRGKRGRDDDDDDDDEESDDAYDEVGND,49,0.017593,0.591837,P31376,verified,0.539053,disordered
1,YAL011W,SWC3,626,169,261,RLFILKNDKIEQKWQDEQELKKKEKELKRKNDAEAKRLRMEERKRQ...,93,0.052936,0.559140,P31376,verified,0.098538,helix
2,YAL011W,SWC3,626,361,424,KTAATEPEPKKADDENAEKQQSKEAKTTAESTQVDVKKEEEDVKEK...,64,0.007812,0.500000,P31376,verified,0.089231,disordered
3,YAL013W,DEP1,406,84,159,ESLKRPHEDEKEAIDEAKKMKVPGENEDESKEEEKSQELEEAIDSK...,76,0.138444,0.592105,P31385,verified,0.219047,disordered
4,YAL019W,FUN30,1132,463,538,ERETKRIRNTTKPKVVEDEDDDVDLEAIDDELPQSEHEDDDYEEED...,76,0.269474,0.644737,P31380,verified,0.498598,disordered


In [20]:
df_hc = df_hc.rename(columns={'region.seq': 'region_seq', 'region.len': 'len_region'})

In [21]:
df_hc['norm_hydrop'] = df_hc.apply(lambda row: append_hydropathy(row), axis=1)
df_hc['norm_nc'] = df_hc.apply(lambda row: append_norm_nc(row), axis=1)
df_hc['uversky_pred'] = df_hc.apply(lambda row: append_uversky_pred(row), axis=1)
df_hc['uversky_correctness'] = df_hc.apply(lambda row:append_uversky_correctness(row), axis=1)

In [22]:
df_hc.to_csv('../../data/uversky/uversky_hc_trimmed.csv', index=False)

## Accuracy

In [23]:
cnf_matrix = np.array([[0, 0], [0, 0]])
cnf_matrix[0, 0] = len(df_hc[(df_hc.label == 'disordered') & (df_hc.uversky_pred == 'disordered')])
cnf_matrix[0, 1] = len(df_hc[(df_hc.label == 'disordered') & (df_hc.uversky_pred == 'helix')])
cnf_matrix[1, 0] = len(df_hc[(df_hc.label == 'helix') & (df_hc.uversky_pred == 'disordered')])
cnf_matrix[1, 1] = len(df_hc[(df_hc.label == 'helix') & (df_hc.uversky_pred == 'helix')])

In [24]:
cnf_matrix

array([[354,   0],
       [326,   1]])

In [25]:
1 - np.sum(df_hc['uversky_correctness']) / len(df_hc)

0.4787077826725403

In [26]:
len(df_hc[(df_hc.label == 'helix') & (df_hc.uversky_pred == 'disordered')]) / len(df_hc[df_hc.label == 'helix'])

0.9969418960244648

In [27]:
len(df[(df.label == 'helix') & (df.uversky_pred == 'helix')]) / len(df[df.uversky_pred == 'helix'])

0.8042553191489362

In [28]:
len(df[(df.label == 'disordered') & (df.uversky_pred == 'disordered')]) / len(df[df.uversky_pred == 'disordered'])

0.7353846153846154

In [29]:
len(df[(df.label == 'disordered') & (df.uversky_pred == 'helix')]) / len(df[df.uversky_pred == 'helix'])

0.19574468085106383